In [1]:
import numpy as np
import tensorflow as tf

/Users/sanghyub/anaconda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/sanghyub/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/Users/sanghyub/anaconda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/sanghyub/anaconda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
char_arr = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']

In [3]:
num_dic = {n:i for i, n in enumerate(char_arr)}

In [5]:
dic_len = len(num_dic)

In [6]:
dic_len

26

In [7]:
seq_data = ['word','wood','deep','dive','cold','cool','load','love','kiss','kind']

In [10]:
seq_data[0][-1]

'd'

In [26]:
def make_batch(seq_data):
    input_batch = []
    target_batch = []
    
    for seq in seq_data:
        input = [num_dic[n] for n in seq[:-1]]
        target = num_dic[seq[-1]]
        #입력값을 원핫인코딩
        input_batch.append(np.eye(dic_len)[input])
        target_batch.append(target)
    return input_batch, target_batch

In [15]:
learning_rate = 0.001
n_hidden = 128
total_epoch = 30
n_step = 3
n_input = n_class = dic_len

In [17]:
X = tf.placeholder(tf.float32, [None, n_step, n_input])
Y = tf.placeholder(tf.int32, [None]) #인덱스 숫자를 그대로 사용할 것이므로 1차원 배열을 사용한다.

W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

In [19]:
cell1 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
cell2 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob = 0.5)
cell2 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)

In [20]:
multi_cell = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2])
outputs, states = tf.nn.dynamic_rnn(multi_cell, X, dtype=tf.float32)

In [21]:
outputs = tf.transpose(outputs, [1,0,2])
outputs = outputs[-1]
model = tf.matmul(outputs, W) + b

In [22]:
cost = tf.reduce_mean(
    tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model, labels=Y))

In [23]:
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [24]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [27]:
input_batch, target_batch = make_batch(seq_data)

In [28]:
for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost],
                      feed_dict={X:input_batch, Y:target_batch})
    print('Epoch', '%04d' % (epoch+1),
         'cost=', '{:.6f}'.format(loss))
print('Complete!')

Epoch 0001 cost= 2.566197
Epoch 0002 cost= 2.477108
Epoch 0003 cost= 2.389535
Epoch 0004 cost= 2.302232
Epoch 0005 cost= 2.214286
Epoch 0006 cost= 2.125178
Epoch 0007 cost= 2.034795
Epoch 0008 cost= 1.943469
Epoch 0009 cost= 1.851990
Epoch 0010 cost= 1.761550
Epoch 0011 cost= 1.673579
Epoch 0012 cost= 1.589380
Epoch 0013 cost= 1.509551
Epoch 0014 cost= 1.433443
Epoch 0015 cost= 1.359050
Epoch 0016 cost= 1.283610
Epoch 0017 cost= 1.204709
Epoch 0018 cost= 1.121270
Epoch 0019 cost= 1.034024
Epoch 0020 cost= 0.945459
Epoch 0021 cost= 0.859345
Epoch 0022 cost= 0.779801
Epoch 0023 cost= 0.709804
Epoch 0024 cost= 0.649704
Epoch 0025 cost= 0.597210
Epoch 0026 cost= 0.549208
Epoch 0027 cost= 0.503518
Epoch 0028 cost= 0.459238
Epoch 0029 cost= 0.416613
Epoch 0030 cost= 0.377171
Complete!


In [30]:
prediction = tf.cast(tf.argmax(model,1), tf.int32)
prediction_check = tf.equal(prediction, Y)
accuracy = tf.reduce_mean(tf.cast(prediction_check, tf.float32))

In [31]:
predict, accuracy_val = sess.run([prediction, accuracy],
                                feed_dict = {X:input_batch, Y:target_batch})

In [33]:
predict_word = []
for idx, val in enumerate(seq_data):
    last_char = char_arr[predict[idx]]
    predict_word.append(val[:3]+last_char)

In [35]:
print('입력값:', [w[:3] + ' ' for w in seq_data])
print('predict:', predict_word)
print('Accuracy:', accuracy_val)

입력값: ['wor ', 'woo ', 'dee ', 'div ', 'col ', 'coo ', 'loa ', 'lov ', 'kis ', 'kin ']
predict: ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load', 'love', 'kiss', 'kind']
Accuracy: 1.0
